# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
import os
import joblib

from azureml.train.automl import AutoMLConfig
from pprint import pprint # Used in printing automl model parameters
from azureml.core import Model # Used to get model information

## Dataset

### Overview

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
from azureml.data.dataset_factory import TabularDatasetFactory

data_loc = "https://raw.githubusercontent.com/khalidw/Capstone-Project-Azure-Machine-Learning-Engineer/master/divorce.csv"
ds = TabularDatasetFactory.from_delimited_files(data_loc)
data = ds.to_pandas_dataframe().dropna()

In [3]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'CapstoneExp'

experiment=Experiment(ws, experiment_name)

In [4]:
# Compute name should contain only letters, digits, hyphen and should be 2-16 charachters long
compute_name = "optCompute"
try:
    trainCluster = ComputeTarget(ws, compute_name)
    print(f"{compute_name} exists already")
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=5)
    trainCluster = ComputeTarget.create(ws, compute_name, compute_config)
trainCluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

In [5]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
        task='classification',
        compute_target=trainCluster,
        training_data=ds,
        label_column_name='Class',
        n_cross_validations=5,
        **automl_settings
)

In [6]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Running on remote.


## Run Details

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.m

{'runId': 'AutoML_5413ead1-f09b-49a8-8702-9e742b4a372d',
 'target': 'optCompute',
 'status': 'Completed',
 'startTimeUtc': '2021-01-09T12:32:25.2752Z',
 'endTimeUtc': '2021-01-09T13:05:15.436654Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'optCompute',
  'AMLSettingsJsonString': '{"path":null,"name":"CapstoneExp","subscription_id":"2c48c51c-bd47-40d4-abbe-fb8eabd19c8c","resource_group":"aml-quickstarts-134086","workspace_name":"quick-starts-ws-134086","region":"southcentralus","compute_target":"optCompute","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_classes":null,"featuriz

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [8]:
# Get best run and model
best_run, fitted_model = remote_run.get_output()

# Print the best run
print(best_run)

# Get all metrics of the best run
best_run_metrics = best_run.get_metrics()

# Print all metrics of the best run
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

Run(Experiment: CapstoneExp,
Id: AutoML_5413ead1-f09b-49a8-8702-9e742b4a372d_26,
Type: azureml.scriptrun,
Status: Completed)
average_precision_score_macro 0.9993464052287582
AUC_weighted 0.9993079584775086
f1_score_macro 0.9881944444444445
AUC_macro 0.9993079584775086
precision_score_weighted 0.9894736842105264
recall_score_weighted 0.9882352941176471
log_loss 0.1022663838491038
f1_score_micro 0.9882352941176471
AUC_micro 0.998961937716263
weighted_accuracy 0.9882352941176471
norm_macro_recall 0.9764705882352942
average_precision_score_weighted 0.9993464052287582
f1_score_weighted 0.9881944444444445
balanced_accuracy 0.9882352941176471
matthews_correlation 0.9777046633277277
precision_score_micro 0.9882352941176471
recall_score_macro 0.9882352941176471
recall_score_micro 0.9882352941176471
accuracy 0.9882352941176471
precision_score_macro 0.9894736842105264
average_precision_score_micro 0.9990086648910179
accuracy_table aml://artifactId/ExperimentRun/dcid.AutoML_5413ead1-f09b-49a8-8702

In [9]:
# Print detailed parameters of the fitted model
def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(
                e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()

print_model(fitted_model)

datatransformer
{'enable_dnn': None,
 'enable_feature_sweeping': None,
 'feature_sweeping_config': None,
 'feature_sweeping_timeout': None,
 'featurization_config': None,
 'force_text_dnn': None,
 'is_cross_validation': None,
 'is_onnx_compatible': None,
 'logger': None,
 'observer': None,
 'task': None,
 'working_dir': None}

SparseNormalizer
{'copy': True, 'norm': 'max'}

GradientBoostingClassifier
{'ccp_alpha': 0.0,
 'criterion': 'mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'deviance',
 'max_depth': 3,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 0.08736842105263157,
 'min_samples_split': 0.15052631578947367,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 25,
 'n_iter_no_change': None,
 'presort': 'deprecated',
 'random_state': None,
 'subsample': 0.8105263157894737,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}



In [10]:
#TODO: Save the best model
myModel = best_run.register_model(model_path='outputs/model.pkl', model_name='capstoneModel_automl',
                        tags={'Training context':'Auto ML'},
                        properties={'Accuracy': best_run_metrics['accuracy']})

print(myModel)

Model(workspace=Workspace.create(name='quick-starts-ws-134086', subscription_id='2c48c51c-bd47-40d4-abbe-fb8eabd19c8c', resource_group='aml-quickstarts-134086'), name=capstoneModel_automl, id=capstoneModel_automl:1, version=1, tags={'Training context': 'Auto ML'}, properties={'Accuracy': '0.9882352941176471'})


In [19]:
# List registered models to verify if model has been saved
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

HyperDrive_HighAccuracy version: 1
	 Accuracy : 0.9803921568627451
	 Regularization Strength : 2.0
	 Max Iterations : 150


capstoneModel_automl version: 1
	 Training context : Auto ML
	 Accuracy : 0.9882352941176471




## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [12]:
# Download scoring file 
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'scoreScript.py')

# Download environment file
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'envFile.yml')

In [13]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script='scoreScript.py',
                                    environment=best_run.get_environment())

# deploy
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(ws, "myservice", [myModel], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

print(service.scoring_uri)

print(service.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running...........................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://40d59825-c328-429e-b6f6-2ff0959dbdb5.southcentralus.azurecontainer.io/score
http://40d59825-c328-429e-b6f6-2ff0959dbdb5.southcentralus.azurecontainer.io/swagger.json


In [14]:
import json

#Import test data
test_df = data.sample(5) # data is the pandas dataframe of the original data
label_df = test_df.pop('Class')

test_sample = json.dumps({'data': test_df.to_dict(orient='records')})

print(test_sample)

{"data": [{"Atr1": 0, "Atr2": 0, "Atr3": 0, "Atr4": 0, "Atr5": 0, "Atr6": 0, "Atr7": 0, "Atr8": 0, "Atr9": 0, "Atr10": 0, "Atr11": 0, "Atr12": 0, "Atr13": 0, "Atr14": 0, "Atr15": 0, "Atr16": 0, "Atr17": 0, "Atr18": 0, "Atr19": 0, "Atr20": 0, "Atr21": 0, "Atr22": 0, "Atr23": 0, "Atr24": 0, "Atr25": 0, "Atr26": 0, "Atr27": 0, "Atr28": 0, "Atr29": 0, "Atr30": 0, "Atr31": 4, "Atr32": 4, "Atr33": 3, "Atr34": 4, "Atr35": 0, "Atr36": 0, "Atr37": 4, "Atr38": 0, "Atr39": 1, "Atr40": 0, "Atr41": 1, "Atr42": 0, "Atr43": 0, "Atr44": 0, "Atr45": 0, "Atr46": 1, "Atr47": 0, "Atr48": 4, "Atr49": 1, "Atr50": 1, "Atr51": 4, "Atr52": 2, "Atr53": 2, "Atr54": 2}, {"Atr1": 0, "Atr2": 0, "Atr3": 0, "Atr4": 0, "Atr5": 0, "Atr6": 0, "Atr7": 0, "Atr8": 0, "Atr9": 0, "Atr10": 0, "Atr11": 0, "Atr12": 1, "Atr13": 0, "Atr14": 0, "Atr15": 0, "Atr16": 0, "Atr17": 1, "Atr18": 0, "Atr19": 0, "Atr20": 0, "Atr21": 1, "Atr22": 0, "Atr23": 0, "Atr24": 0, "Atr25": 0, "Atr26": 0, "Atr27": 0, "Atr28": 0, "Atr29": 0, "Atr30": 

TODO: In the cell below, send a request to the web service you deployed to test it.

In [15]:
import requests # Used for http post request

# Set the content type
headers = {'Content-type': 'application/json'}


response = requests.post(service.scoring_uri, test_sample, headers=headers)

In [16]:
# Print results from the inference
print(response.text)

"{\"result\": [0, 0, 1, 1, 0]}"


In [17]:
# Print original labels
print(label_df)

165    0
84     0
68     1
33     1
105    0
Name: Class, dtype: int64


TODO: In the cell below, print the logs of the web service and delete the service

In [18]:
print(service.get_logs())

2021-01-09T13:14:22,819121841+00:00 - rsyslog/run 
2021-01-09T13:14:22,819151643+00:00 - gunicorn/run 
2021-01-09T13:14:22,819154643+00:00 - iot-server/run 
2021-01-09T13:14:22,852993389+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
EdgeHubC

In [20]:
service.delete()